In [2]:
from aqua.graphics import plot_hovmoller
from teleconnections import mjo_hovmoller
from aqua import Reader

In [3]:
%reload_ext autoreload
%autoreload 2

MJO is a teleconnection that require daily aggregated data.
We specify the teleconnection name and the model/exp/source to analyze.

In [4]:
from aqua import inspect_catalogue

inspect_catalogue(model='ERA5', exp='era5')

Sources available in catalogue for model ERA5 and exp era5:


['monthly', 'daily', 'hourly']

In [5]:
telecname = 'MJO'

model = 'ERA5'
exp = 'era5'
source = 'daily'

The Reader class is user to retrieve the data, with standard resolution `r025` and the `daily` frequency.

In [6]:
reader = Reader(model=model,exp=exp,source=source, regrid='r025')
data = reader.retrieve()

2024-03-27 16:37:58 :: Reader :: WARNING  -> DATA_PATH parameter is required but is missing, setting to default /pfs/lustrep3/projappl/project_465000454/data/AQUA/datasets


2024-03-27 16:38:04 :: Reader :: ERROR    -> Variable time_bnds has no units!


In order to plot the data we subselect a time slice of data, then we use the `Reader` methods to obtain the correctly formatted data for the plot

In [7]:
sel = {'time': slice('2021-01-01','2021-06-30')}

data_sel = data.sel(**sel)
data_sel = reader.timmean(data_sel, freq='daily')
data_sel = reader.regrid(data_sel)

Finally the data are smoothed with a 5 days rolling window, selecting the correct variable and ready to be plotted.

In [8]:
data_sel

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 181, lat: 720, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2021-01-01 2021-01-02 ... 2021-06-30
  * lat      (lat) float64 6kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    mtntrf   (time, lat, lon) float64 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Sat Nov 18 14:40:07 2023: cdo -O -s -f nc4 -z zip -daymean ...
    frequency:    day
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...

In [7]:
data_smooth = mjo_hovmoller(data_sel, telecname=telecname, loglevel='DEBUG', day_window=None)

2024-03-27 16:25:03 :: MJO :: INFO     -> No namelist provided. Trying to load default namelist.
2024-03-27 16:25:04 :: MJO :: INFO     -> Subselecting var mtntrf
2024-03-27 16:25:04 :: MJO :: INFO     -> Subselecting var mtntrf
2024-03-27 16:25:04 :: MJO :: WARNING  -> Flipping the sign of the variable mtntrf.


To plot we use the `plot_hovmoller` function from the framework

In [9]:
plot_hovmoller(data_smooth, dim='lat', invert_axis=True, nlevels=21, loglevel='INFO', sym=True, cmap='PuOr')

2024-03-27 16:11:46 :: Hovmoller :: INFO     -> Averaging over dimension: lat


From the sate Dataset we can smooth and plot another variable, overwriting the default variable from the MJO namelist.